In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
from llama_cpp.llama import Llama, LlamaGrammar


In [2]:
model=Llama.from_pretrained(
            repo_id="NousResearch/Hermes-3-Llama-3.1-8B-GGUF",
            filename="*.Q8_0.gguf",
            n_batch=1024,
            n_ctx=10000,
            n_gpu_layers=-1,
        )


llama_load_model_from_file: using device Metal (Apple M2 Pro) - 21845 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /Users/benedikt/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.1-8B-GGUF/snapshots/307a5dfb59aa38d88b6cfd32f44b8ad7c1da9fb8/./Hermes-3-Llama-3.1-8B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.1 8B
llama_model_loader: - kv   3:                       general.organization str              = NousResearch
llama_model_loader: - kv   4:                           general.basename str              = Hermes-3-Llama-3.1
llama_model_loa

In [19]:
import sys
import os

sys.path.insert(0, os.path.abspath("../.."))
sys.path.append(os.path.abspath("../../backend"))
sys.path.append(os.path.abspath(""))


from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.model import *
from backend.ontology import *
from backend.datasetmatcher import *
from backend.explorative_support import *

store = SPARQLStore(
    "http://localhost:7012/",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
topic_man = TopicModelling(ontology_manager)


In [51]:

import pydantic
from llama_cpp_agent.gbnf_grammar_generator.gbnf_grammar_from_pydantic_models import generate_gbnf_grammar_from_pydantic_models
import json
class Constraint(pydantic.BaseModel):
    property: str
    value: str
    modifier: str
    
class Entity(pydantic.BaseModel):
    identifier: str
    type: str
    constraints: list[Constraint]
    
class Relation(pydantic.BaseModel):
    entity:str
    relation:str
    target:str

class EntitiesRelations(pydantic.BaseModel):
    relations: list[Relation]
    entities: list[Entity]

gbnf_erl = generate_gbnf_grammar_from_pydantic_models([EntitiesRelations], "EntitiesRelations", add_inner_thoughts=False)
print(gbnf_erl)
grammar_erl = LlamaGrammar.from_string(gbnf_erl)
grammar_erl._grammar

root ::= entities-relations
entities-relations ::= (" "| "\n") "{" ws "\"EntitiesRelations\""  ": " ws grammar-models
grammar-models ::= entities-relations-grammar-model
entities-relations-grammar-model ::= "\"EntitiesRelations\"" "," ws "\"None\"" ": " entities-relations

entities-relations ::= "{"  ws "\"relations\"" ": " entities-relations-relations ","  ws "\"entities\"" ": " entities-relations-entities  ws "}"
relation ::= "{"  ws "\"entity\"" ": " string ","  ws "\"relation\"" ": " string ","  ws "\"target\"" ": " string  ws "}"
entities-relations-relations ::= "[" ws (relation)? ("," ws relation)* ws "]" 
entity ::= "{"  ws "\"identifier\"" ": " string ","  ws "\"type\"" ": " string ","  ws "\"constraints\"" ": " entity-constraints  ws "}"
constraint ::= "{"  ws "\"property\"" ": " string ","  ws "\"value\"" ": " string ","  ws "\"modifier\"" ": " string  ws "}"
entity-constraints ::= "[" ws (constraint)? ("," ws constraint)* ws "]" 
entities-relations-entities ::= "[" ws (entit

'root ::= entities-relations\nentities-relations ::= (" "| "\\n") "{" ws "\\"EntitiesRelations\\""  ": " ws grammar-models\ngrammar-models ::= entities-relations-grammar-model\nentities-relations-grammar-model ::= "\\"EntitiesRelations\\"" "," ws "\\"None\\"" ": " entities-relations\n\nentities-relations ::= "{"  ws "\\"relations\\"" ": " entities-relations-relations ","  ws "\\"entities\\"" ": " entities-relations-entities  ws "}"\nrelation ::= "{"  ws "\\"entity\\"" ": " string ","  ws "\\"relation\\"" ": " string ","  ws "\\"target\\"" ": " string  ws "}"\nentities-relations-relations ::= "[" ws (relation)? ("," ws relation)* ws "]" \nentity ::= "{"  ws "\\"identifier\\"" ": " string ","  ws "\\"type\\"" ": " string ","  ws "\\"constraints\\"" ": " entity-constraints  ws "}"\nconstraint ::= "{"  ws "\\"property\\"" ": " string ","  ws "\\"value\\"" ": " string ","  ws "\\"modifier\\"" ": " string  ws "}"\nentity-constraints ::= "[" ws (constraint)? ("," ws constraint)* ws "]" \nenti

In [9]:
# System prompt describes information given to all conversations
ERL_PROMPT_SYSTEM = """
Return all the entity relations within the prompt in the form of JSON output. The output should be a list of all entities and their constraints, as well as the relations between them. Make sure to include all entities and targets in the list of entities.
"""
ERL_SAMPLE = EntitiesRelations(
    relations=[Relation(entity="person 1", relation="author", target="work 1")],
    entities=[
        Entity(
            identifier="person 1",
            type="person",
            constraints=[
                Constraint(property="birth_date", value="1990", modifier="greater_than")
            ],
        ),
        Entity(identifier="work 1", type="work", constraints=[]),
    ],
)
# Example prompt demonstrating the output we are looking for
ERL_PROMPT_EXAMPLE = (
    "the birth place of an author of a work where the author is born after 1990",
    ERL_SAMPLE.model_dump_json(),
)

In [22]:
query = "a person who created a work who is taller than 180cm"

response = model.create_chat_completion(
    grammar=grammar_erl,
    messages=[
        {"role": "system", "content": ERL_PROMPT_SYSTEM},
        {"role": "user", "content": ERL_PROMPT_EXAMPLE[0]},
        {"role": "assistant", "content": ERL_PROMPT_EXAMPLE[1]},
        {"role": "user", "content": query},
    ],
    max_tokens=-1,
)
response_msg = response["choices"][0]["message"]["content"]

Llama.generate: 152 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    2567.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   130 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16264.31 ms /   131 tokens


In [23]:
print(response_msg)

{
  "relations": [
    {
      "entity": "person",
      "relation": "creator",
      "target": "work"
    }
  ],
  "entities": [
    {
      "name": "person",
      "constraints": [
        {
          "property": "height",
          "value": "180cm",
          "modifier": "greater_than"
        }
      ]
    },
    {
      "name": "work",
      "constraints": [
        {
          "property": "creator",
          "value": "person",
          "modifier": "exists"
        }
      ]
    }
  ]
}


In [24]:


erl_llm = EntitiesRelations.model_validate_json(response_msg)
erl_llm

EntitiesRelations(relations=[Relation(entity='person', relation='creator', target='work')], entities=[Entity(name='person', constraints=[Constraint(property='height', value='180cm', modifier='greater_than')]), Entity(name='work', constraints=[Constraint(property='creator', value='person', modifier='exists')])])

In [46]:
class CandidateRelation(Relation):
    score: float


class CandidateConstraint(Constraint):
    score: float
    type: str

class CandidateEntity(pydantic.BaseModel):
    score: float
    type: str


class Candidates(pydantic.BaseModel):
    relations: list[CandidateRelation]
    entities: list[CandidateEntity]
    constraints: list[CandidateConstraint]


gbnf_candidates = generate_gbnf_grammar_from_pydantic_models(
    [Candidates], "Candidates", add_inner_thoughts=False
)
grammer_candidates = LlamaGrammar.from_string(gbnf_candidates)

In [47]:
example_limit = 3
relation_candidates: list[CandidateRelation] = []
constraint_candidates: list[CandidateConstraint] = []
entity_candidates: list[CandidateEntity] = []
for relation in erl_llm.relations:
    top_results = topic_man.search_fuzzy(
        query=FuzzyQuery(
            q=f"A {relation.entity} is {relation.relation} of {relation.target}",
            limit=example_limit,
            type=RETURN_TYPE.LINK,
            relation_type=RELATION_TYPE.INSTANCE,
        )
    )
    relation_candidates.extend(
        [
            CandidateRelation(
                entity=res.link.from_subject.label,
                relation=res.link.label,
                target=res.link.to_subject.label,
                score=res.score,
            )
            for res in top_results.results
        ]
    )

for entity in erl_llm.entities:
    top_results = topic_man.search_fuzzy(
        query=FuzzyQuery(
            q=f"A {entity.name}",
            limit=example_limit,
            type=RETURN_TYPE.SUBJECT,
            relation_type=RELATION_TYPE.INSTANCE,
        )
    )
    entity_candidates.extend(
        [
            CandidateEntity(
                type=res.subject.label,
                score=res.score,
            )
            for res in top_results.results
        ]
    )
    for constraint in entity.constraints:
        top_results = topic_man.search_fuzzy(
            query=FuzzyQuery(
                q=f"The {constraint.property} of is {constraint.modifier} {constraint.value}",
                limit=example_limit,
                type=RETURN_TYPE.LINK,
                relation_type=RELATION_TYPE.PROPERTY,
                
            )
        )
        constraint_candidates.extend(
            [
                CandidateConstraint(
                    property=res.link.label,
                    value=constraint.value,
                    modifier=constraint.modifier,
                    score=res.score,
                    type=res.link.to_proptype,
                )
                for res in top_results.results
            ]
        )
candidates = Candidates(
    relations=relation_candidates,
    entities=entity_candidates,
    constraints=constraint_candidates,
)

In [48]:
candidates

Candidates(relations=[CandidateRelation(entity='person', relation='created', target='work', score=0.2854456066733607), CandidateRelation(entity='work', relation='auteur', target='person', score=0.2991304962968474), CandidateRelation(entity='work', relation='composer', target='person', score=0.300885253116127)], entities=[CandidateEntity(score=0.3762859689649488, type='person'), CandidateEntity(score=0.3820784245203156, type='businessperson'), CandidateEntity(score=0.4021959752199751, type='Academic Person'), CandidateEntity(score=0.28001735523805793, type='work'), CandidateEntity(score=0.3370414595099054, type='written work'), CandidateEntity(score=0.3554376988089626, type='unit of work')], constraints=[CandidateConstraint(property='height (cm)', value='180cm', modifier='greater_than', score=0.4108847133315189, type='<http://dbpedia.org/datatype/centimetre>'), CandidateConstraint(property='height (mm)', value='180cm', modifier='greater_than', score=0.48760438819498353, type='<http://db

In [ ]:


RAG_PROMPT_EXAMPLE_CANDIDATES = Candidates(
    relations=[
        CandidateRelation(
            entity="person 1", relation="author", target="work 1", score=0.9
        )
    ],
    entities=[
        CandidateEntity(type="person", score=0.9),
        CandidateEntity(type="composer", score=0.7),
        CandidateEntity(type="work", score=0.9),
        CandidateEntity(type="song", score=0.6),
    ],
    constraints=[
        CandidateConstraint(
            property="birth date",
            value="1990",
            modifier="greater_than",
            score=0.9,
            type="sko:DateTime",
        ),
        CandidateConstraint(
            property="death year",
            value="180cm",
            modifier="greater_than",
            score=0.8,
            type="sko:DateTime",
        ),
    ],
)

RAG_PROMPT_SYSTEM = f"""
Return all the entities, relations, and constraints within the prompt in the form of JSON output. The output should be a list of all entities and their constraints, as well as the relations between them. Make sure to include only the entities and targets in the following list of candidates:
{RAG_PROMPT_EXAMPLE_CANDIDATES.model_dump_json()}
"""

RAG_PROMPT_EXPECTED_OUTPUT = EntitiesRelations(
    relations=[Relation(entity="person 1", relation="author", target="work 1")],
    entities=[
        Entity(
            identifier="person 1",
            type="person",
            constraints=[
                Constraint(
                    property="birth date",
                    value="1.1.1990",
                    modifier="greater_than",
                )
            ],
        ),
        Entity(identifier="work 1", type="work", constraints=[]),
    ],
)
RAG_PROMPT_EXAMPLE = (
    "the birth place of an author of a work where the author is born after 1990",
    RAG_PROMPT_EXAMPLE_CANDIDATES.model_dump_json(),
    RAG_PROMPT_EXPECTED_OUTPUT.model_dump_json(),
)

In [53]:

response = model.create_chat_completion(
    grammar=grammar_erl,
    messages=[
        {"role": "system", "content": RAG_PROMPT_SYSTEM},
        {"role": "user", "content": RAG_PROMPT_EXAMPLE[0]},
        {"role": "assistant", "content": RAG_PROMPT_EXAMPLE[1]},
        {"role": "user", "content": query},
    ],
    max_tokens=-1,
)
response_msg = response["choices"][0]["message"]["content"]

Llama.generate: 6 prefix-match hit, remaining 229 prompt tokens to eval
llama_perf_context_print:        load time =    2567.32 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   229 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   160 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   15569.51 ms /   389 tokens


In [55]:
print(response_msg)

{
  "relations": [
    {
      "entity": "person 1",
      "relation": "creator",
      "target": "work 1"
    }
  ],
  "entities": [
    {
      "identifier": "person 1",
      "type": "person",
      "constraints": [
        {
          "property": "height",
          "value": "180cm",
          "modifier": "greater_than"
        }
      ]
    },
    {
      "identifier": "work 1",
      "type": "work",
      "constraints": [
        {
          "property": "creator",
          "value": "person 1",
          "modifier": "same_as"
        }
      ]
    }
  ]
}
